In [1]:
import pandas as pd
import numpy as np
from kafka import KafkaProducer
from kafka import KafkaConsumer
import json
import time
import logging
import pickle
from pyspark.sql.functions import *
import csv

In [2]:
topic_name = "clickstream_data"
kafka_broker = "localhost:9092"

In [37]:
def create_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers=kafka_broker, 
            value_serializer=lambda x:json.dumps(x).encode('utf-8'),
            key_serializer=lambda x:json.dumps(x).encode('utf-8'),
        )
    except Exception as e:
        logging.exception("Couldn't create the producer")
        producer = None
    
    return producer

def create_consumer(topic_name):
    try:
        consumer = KafkaConsumer(
        topic_name,
        bootstrap_servers=kafka_broker,
        auto_offset_reset="earliest",
        enable_auto_commit=True,
        value_deserializer=lambda x: json.loads(x.decode("utf-8")))
    except Exception as e:
        logging.exception("Couldn't create the consumer")
        consumer = None
    return consumer

In [38]:
producer = create_producer()

In [39]:
with open("./public data/data_a_hidden_10.csv", 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader, None)
    for row in csv_reader:
        message = ','.join(row)
        print(message)
        producer.send(topic_name, value=message)
        time.sleep(2)
        producer.flush()
        break

2333000637,A,VH356862,Directions,Enter Item,,2017-02-02 16:27:32.820


In [3]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [4]:
data_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_broker) \
        .option("subscribe", topic_name) \
        .option("startingOffsets", "earliest") \
        .load() 
data_df_csv = data_df.selectExpr("CAST(value as STRING)").withColumn("value", expr("substring(value, 2, length(value)-2)"))

In [5]:
data_schema_string = 'STUDENTID STRING, Block STRING, AccessionNumber STRING, ItemType STRING, Observable STRING, ExtendedInfo STRING, EventTime TIMESTAMP'

data_df_csv = data_df_csv \
        .select(from_csv(col("value"), data_schema_string) \
                .alias("data"))


data_df_csv = data_df_csv.select("data.*")

In [6]:
data_df_csv.printSchema()

root
 |-- STUDENTID: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- AccessionNumber: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Observable: string (nullable = true)
 |-- ExtendedInfo: string (nullable = true)
 |-- EventTime: timestamp (nullable = true)



In [12]:
query1.stop()

In [7]:
q1name = "query"
stream_writer1 = (data_df_csv.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()
spark.sql(f"SELECT * from {query1.name}").toPandas() 

,STUDENTID,Block,AccessionNumber,ItemType,Observable,ExtendedInfo,EventTime
